## 1. Preliminary

### 1.1 Libraries

In [1]:
import pandas as pd
import numpy as np

# Options pandas
pd.options.mode.chained_assignment = None

### 1.2 Data

In [2]:
# Constants for average energy needs
DAILY_KCAL_NEED_FEMALE = 2100
DAILY_KCAL_NEED_MALE = 2550
DAILY_KCAL_NEED_AVERAGE = (DAILY_KCAL_NEED_FEMALE + DAILY_KCAL_NEED_MALE) / 2
YEARLY_KCAL_NEED = DAILY_KCAL_NEED_AVERAGE * 365

## 2. Data preparation

In [3]:
# Function to import data with optional filters
def import_data(file_path, year_column, value_column, year_filter=None, value_filter=None):
    data = pd.read_csv(file_path)
    if year_filter is not None:
        data = data[data[year_column] == year_filter]
    if value_filter is not None:
        data = data[data[value_column] > value_filter]
    return data

In [4]:
# Data Importation
def load_data():
    population_2017 = import_data("population.csv", "Année", "Valeur", year_filter=2017, value_filter=0.0)
    malnourished_2017 = import_data("sous_nutrition.csv", "Année", "Valeur", year_filter="2016-2018")
    food_availability = pd.read_csv("dispo_alimentaire.csv")
    donations_2013 = import_data("aide_alimentaire.csv", "Année", "Valeur", year_filter=2013)
    return population_2017, malnourished_2017, food_availability, donations_2013

In [5]:
# Handling missing data
def handle_missing_values(data):
    if "Valeur" in data.columns:
        # If the “Value” column contains strings, replace them with 0
        if data["Valeur"].dtype == 'object':
            data.loc[data["Valeur"] == "<0.1", "Valeur"] = 0
            # Convert column to float type
            data["Valeur"] = data["Valeur"].astype(float)
        # Replace missing values with 0
        data["Valeur"].fillna(0, inplace=True)
    return data

In [6]:
# Data type conversion
def convert_data_types(data):
    if "Année" in data.columns:
        data["Année"] = data["Année"].astype(str)
    if "Valeur" in data.columns:
        data["Valeur"] = pd.to_numeric(data["Valeur"], errors='coerce')
    return data

In [7]:
# Data preparation
def prepare_data():
    population_2017, malnourished_2017, food_availability, donations_2013 = load_data()
    population_2017 = handle_missing_values(population_2017)
    malnourished_2017 = handle_missing_values(malnourished_2017)
    food_availability = handle_missing_values(food_availability)
    donations_2013 = handle_missing_values(donations_2013)
    population_2017 = convert_data_types(population_2017)
    malnourished_2017 = convert_data_types(malnourished_2017)
    food_availability = convert_data_types(food_availability)
    donations_2013 = convert_data_types(donations_2013)
    return population_2017, malnourished_2017, food_availability, donations_2013

In [8]:
# Loading prepared data
population_2017, malnourished_2017, food_availability, donations_2013 = prepare_data()

### 2.1 Unit Conversions

In [9]:
population_2017["Valeur"] = population_2017["Valeur"] / 1000
food_availability["Disponibilité alimentaire (Kcal/personne/jour)"] *= 365
donations_2013["Valeur"] = donations_2013["Valeur"] / 1000

### 2.2 Column names

In [10]:
population_2017.rename(columns={"Valeur": "Population_totale"}, inplace=True)
malnourished_2017.rename(columns={"Valeur": "Population_sous_alimentée", "Année": "Intervalle"}, inplace=True)
donations_2013.rename(columns={"Pays bénéficiaire": "Zone", "Valeur": "Quantité"}, inplace=True)
food_availability.rename(columns={"Disponibilité alimentaire (Kcal/personne/jour)": "Kcal_personne_an", "Disponibilité intérieure": "Disponibilité_intérieure_milliers_tonnes"}, inplace=True)

# Part I - Global Malnutrition

### Section 1: Proportion of Malnourished people in 2017<a class="anchor" id="question1"></a>

In [11]:
# Merge of Global Population and Malnourished Population
malnourished_merged = pd.merge(population_2017, malnourished_2017, how="left", on="Zone")
malnourished_merged = malnourished_merged[["Zone", "Population_totale", "Population_sous_alimentée"]]

### 1.1 Population in 2017<a class="anchor" id="section_1_1"></a>

In [12]:
global_population = malnourished_merged["Population_totale"].sum() / 1000
print("In 2017, the global population was", round(global_population, 2), "billion.")

In 2017, the global population was 7.55 billion.


### 1.2 Malnourished Population in 2017<a class="anchor" id="section_1_2"></a>

In [13]:
global_undernourished_population = malnourished_merged["Population_sous_alimentée"].sum() / 1000
print("In 2017, the global malnourished population was", round(global_undernourished_population, 2), "billion.")

In 2017, the global malnourished population was 0.54 billion.


### 1.3 Proportion of the Population Malnourished in 2017<a class="anchor" id="section_1_3"></a>

In [14]:
proportion_undernourished = (global_undernourished_population / global_population) * 100
print("In 2017, the proportion of people in a state of malnutrition is", round(proportion_undernourished, 2), "%")

In 2017, the proportion of people in a state of malnutrition is 7.1 %


### Section 2. Theoretical Number of People Who Could be Fed in 2017<a class="anchor" id="question2"></a>

### 2.1 Kcal/person/day by Country<a class="anchor" id="section_2_1"></a>

In [15]:
food_availability_country = food_availability.groupby(["Zone"])["Kcal_personne_an"].sum().reset_index()

In [16]:
# Merge of population by country with food availability by country
food_merged = pd.merge(food_availability_country, population_2017, how="left", on="Zone")
food_merged = food_merged[["Zone", "Kcal_personne_an", "Population_totale"]]

In [17]:
# Displaying merged data
display(food_merged.reset_index(drop=True).head().round(2))

,Zone,Kcal_personne_an,Population_totale
0,Afghanistan,761755.0,36.30
1,Afrique du Sud,1102300.0,57.01
2,Albanie,1163620.0,2.88
3,Algérie,1201945.0,41.39
4,Allemagne,1278595.0,82.66


### 2.2 Proportion of Population that could be fed, by Country<a class="anchor" id="section_2_2"></a>

In [18]:
food_merged["Proportion Fed"] = food_merged["Kcal_personne_an"] / DAILY_KCAL_NEED_AVERAGE

In [19]:
# Displaying proportion of population that could be fed
food_merged.head().round(2)

,Zone,Kcal_personne_an,Population_totale,Proportion Fed
0,Afghanistan,761755.0,36.30,327.64
1,Afrique du Sud,1102300.0,57.01,474.11
2,Albanie,1163620.0,2.88,500.48
3,Algérie,1201945.0,41.39,516.97
4,Allemagne,1278595.0,82.66,549.93


### 2.3 Theoretical Number of People who could be Fed, by Country<a class="anchor" id="section_2_3"></a>

In [20]:
food_merged["Theoretical Fed"] = food_merged["Population_totale"] * food_merged["Proportion Fed"]

In [21]:
# Displaying theoretical number of people who could be fed
food_merged.head().round(2)

,Zone,Kcal_personne_an,Population_totale,Proportion Fed,Theoretical Fed
0,Afghanistan,761755.0,36.30,327.64,11891.93
1,Afrique du Sud,1102300.0,57.01,474.11,27028.75
2,Albanie,1163620.0,2.88,500.48,1443.47
3,Algérie,1201945.0,41.39,516.97,21396.79
4,Allemagne,1278595.0,82.66,549.93,45456.61


In [22]:
# Countries with the most food availability per kcal needed
food_merged = food_merged.sort_values("Theoretical Fed", ascending=False)
display(food_merged.reset_index(drop=True).head().round(2))

,Zone,Kcal_personne_an,Population_totale,Proportion Fed,Theoretical Fed
0,"Chine, continentale",1135880.0,1421.02,488.55,694240.96
1,Inde,895710.0,1338.68,385.25,515727.39
2,États-Unis d'Amérique,1343930.0,325.08,578.03,187910.17
3,Indonésie,1013240.0,264.65,435.80,115335.46
4,Brésil,1190630.0,207.83,512.10,106431.48


### 2.4 Total Theoretical Number of People who could be Fed<a class="anchor" id="section_2_4"></a>

In [23]:
total_available_people = food_merged["Theoretical Fed"].sum() / 1000
print("For 2017, the theoretical number of people who could be fed is", round(total_available_people, 3), "billion.")

For 2017, the theoretical number of people who could be fed is 3284.056 billion.


### Section 3: Theoretical Number of People who could be Fed with a Plant-based Diet<a class="anchor" id="question3"></a>

### 3.1 Kcals/person/year for Plant-based Products, by Country<a class="anchor" id="section_3_1"></a>

In [24]:
plant_based_availability = food_availability[(food_availability["Origine"] == "vegetale")]
plant_merged = pd.merge(plant_based_availability, population_2017, how="left", on="Zone")
plant_merged_country = plant_merged.groupby(["Zone"])["Kcal_personne_an"].sum().reset_index()
plant_merged_country.head()

,Zone,Kcal_personne_an
0,Afghanistan,682915.0
1,Afrique du Sud,924545.0
2,Albanie,804095.0
3,Algérie,1063975.0
4,Allemagne,898265.0


In [25]:
# Merge population and food availability by country
plant_food_merged = pd.merge(plant_merged_country, population_2017, how="left", on="Zone")
plant_food_merged = plant_food_merged[["Zone", "Kcal_personne_an", "Population_totale"]]
plant_food_merged.reset_index(drop=True).head().round(2)

,Zone,Kcal_personne_an,Population_totale
0,Afghanistan,682915.0,36.30
1,Afrique du Sud,924545.0,57.01
2,Albanie,804095.0,2.88
3,Algérie,1063975.0,41.39
4,Allemagne,898265.0,82.66


### 3.2 Proportion of Population that could be Fed with Plant-based Products, by Country<a class="anchor" id="section_3_2"></a>

In [26]:
plant_food_merged["Proportion Fed"] = plant_food_merged["Kcal_personne_an"] / DAILY_KCAL_NEED_AVERAGE
plant_food_merged.head().round(2)

,Zone,Kcal_personne_an,Population_totale,Proportion Fed
0,Afghanistan,682915.0,36.30,293.73
1,Afrique du Sud,924545.0,57.01,397.65
2,Albanie,804095.0,2.88,345.85
3,Algérie,1063975.0,41.39,457.62
4,Allemagne,898265.0,82.66,386.35


### 3.3 Theoretical Number of People who could be Fed with Plant-based Products, by Country<a class="anchor" id="section_3_3"></a>

In [27]:
plant_food_merged["Theoretical Fed"] = plant_food_merged["Population_totale"] * plant_food_merged["Proportion Fed"]
plant_food_merged.head().round(2)

,Zone,Kcal_personne_an,Population_totale,Proportion Fed,Theoretical Fed
0,Afghanistan,682915.0,36.30,293.73,10661.14
1,Afrique du Sud,924545.0,57.01,397.65,22670.14
2,Albanie,804095.0,2.88,345.85,997.48
3,Algérie,1063975.0,41.39,457.62,18940.67
4,Allemagne,898265.0,82.66,386.35,31935.12


In [28]:
# Countries with the most food availability (plant-based products) per kcal needed
plant_food_merged = plant_food_merged.sort_values("Theoretical Fed", ascending=False)
display(plant_food_merged.reset_index(drop=True).head(3).round(2))

,Zone,Kcal_personne_an,Population_totale,Proportion Fed,Theoretical Fed
0,"Chine, continentale",871985.0,1421.02,375.05,532950.40
1,Inde,809935.0,1338.68,348.36,466340.29
2,États-Unis d'Amérique,984770.0,325.08,423.56,137691.92


### 3.4 Total Theoretical Number of People who could be Fed with Plant-based Products<a class="anchor" id="section_3_4"></a>

In [29]:
total_plant_available_people = plant_food_merged["Proportion Fed"].sum() / 1000
print("For the year 2017, the theoretical number of people who could be fed with plant-based products is", round(total_plant_available_people, 2), "billion.")

For the year 2017, the theoretical number of people who could be fed with plant-based products is 62.6 billion.


## Section 4: Use of Domestic Availability<a class="anchor" id="question4"></a>

### 4.1 Global Domestic Availability<a class="anchor" id="section_4_1"></a>

In [30]:
global_domestic_availability = food_availability[(food_availability["Disponibilité_intérieure_milliers_tonnes"] > 0)]
global_domestic_availability = global_domestic_availability["Disponibilité_intérieure_milliers_tonnes"].sum()
print("Global domestic availability is", round(global_domestic_availability), "thousand tonnes.")

Global domestic availability is 9854329 thousand tonnes.


### 4.2 Domestic Availability, by Country<a class="anchor" id="section_4_2"></a>

In [31]:
country_domestic_availability = food_availability.groupby(["Zone"])["Disponibilité_intérieure_milliers_tonnes"].sum().reset_index()
country_domestic_availability.head()

,Zone,Disponibilité_intérieure_milliers_tonnes
0,Afghanistan,13515.0
1,Afrique du Sud,61256.0
2,Albanie,4758.0
3,Algérie,42630.0
4,Allemagne,162275.0


## Section 5: Domestic Availability allocated to Animal Nutrition<a class="anchor" id="section_4_3"></a>

### 5.1 Proportion of Domestic Availability allocated to Animal Nutrition<a class="anchor" id="sub_section_4_3_2"></a>

In [32]:
animal_quantity = food_availability[(food_availability["Aliments pour animaux"] > 0)]
animal_quantity = food_availability["Aliments pour animaux"].sum()
print("The quantity of domestic availability allocated to animal nutrition is", round(animal_quantity), "thousand tonnes.")

The quantity of domestic availability allocated to animal nutrition is 1304245 thousand tonnes.


In [33]:
animal_proportion = (animal_quantity / global_domestic_availability) * 100
print("The proportion of domestic availability allocated to animal nutrition is", round(animal_proportion, 2), "%")

The proportion of domestic availability allocated to animal nutrition is 13.24 %


### 5.2 Proportion of Domestic Availability alloced to Animal Nutrition, by Country<a class="anchor" id="sub_section_4_3_1"></a>

In [34]:
animal_by_country = food_availability[food_availability["Aliments pour animaux"] > 0]
animal_by_country = animal_by_country.groupby(["Zone"])["Aliments pour animaux"].sum().reset_index()

In [35]:
# Merge domestic availability by country with food availability attributed to animal nutrition
avail_prop_animal = pd.merge(country_domestic_availability, animal_by_country, how="left", on="Zone")
avail_prop_animal.reset_index(drop=True).head().round(2)

,Zone,Disponibilité_intérieure_milliers_tonnes,Aliments pour animaux
0,Afghanistan,13515.0,768.0
1,Afrique du Sud,61256.0,5309.0
2,Albanie,4758.0,660.0
3,Algérie,42630.0,4352.0
4,Allemagne,162275.0,30209.0


In [36]:
avail_prop_animal["Proportion Aliments pour animaux"] = avail_prop_animal["Aliments pour animaux"] / avail_prop_animal["Disponibilité_intérieure_milliers_tonnes"]
display(avail_prop_animal.head().round(2))

,Zone,Disponibilité_intérieure_milliers_tonnes,Aliments pour animaux,Proportion Aliments pour animaux
0,Afghanistan,13515.0,768.0,0.06
1,Afrique du Sud,61256.0,5309.0,0.09
2,Albanie,4758.0,660.0,0.14
3,Algérie,42630.0,4352.0,0.10
4,Allemagne,162275.0,30209.0,0.19


In [37]:
# Countries that allocated the most domestic availability to animal nutrition
top_animal_countries = avail_prop_animal.sort_values("Proportion Aliments pour animaux", ascending=False)
top_animal_countries.reset_index(drop=True).head().round(2)

,Zone,Disponibilité_intérieure_milliers_tonnes,Aliments pour animaux,Proportion Aliments pour animaux
0,Danemark,19749.0,7835.0,0.40
1,Arabie saoudite,31274.0,11841.0,0.38
2,Bélarus,27522.0,10204.0,0.37
3,Tadjikistan,7617.0,2775.0,0.36
4,Irlande,10738.0,3758.0,0.35


### Section 6: Domestic Availability allocated to Human Consumption<a class="anchor" id="section_4_4"></a>

### 6.1 Proportion of Domestic Availability attributed to Human Consumption<a class="anchor" id="sub_section_4_4_2"></a>

In [38]:
total_human_quantity = food_availability["Nourriture"].sum()
print("The quantity of domestic availability attributed to human consumption is", round(total_human_quantity), "thousand tonnes.")

The quantity of domestic availability attributed to human consumption is 4876258 thousand tonnes.


In [39]:
proportion_human = (total_human_quantity / global_domestic_availability) * 100
print("The proportion of domestic availability allocated to human consumption is", round(proportion_human, 2), "%")

The proportion of domestic availability allocated to human consumption is 49.48 %


### 6.2 Proportion of Domestic Availability allocated to Human Consumption, by Country<a class="anchor" id="sub_section_4_4_1"></a>

In [40]:
human_by_country = food_availability[food_availability["Nourriture"] > 0]
human_by_country = human_by_country.groupby(["Zone"])["Nourriture"].sum().reset_index()

In [41]:
# Merge domestic availability by country with food availability attributed to human consumption
avail_prop_human = pd.merge(country_domestic_availability, human_by_country, how="left", on="Zone")
avail_prop_human.reset_index(drop=True).head().round(2)

,Zone,Disponibilité_intérieure_milliers_tonnes,Nourriture
0,Afghanistan,13515.0,10735.0
1,Afrique du Sud,61256.0,29812.0
2,Albanie,4758.0,3476.0
3,Algérie,42630.0,31729.0
4,Allemagne,162275.0,79238.0


In [42]:
avail_prop_human["Proportion Nourriture"] = avail_prop_human["Nourriture"] / avail_prop_human["Disponibilité_intérieure_milliers_tonnes"]
avail_prop_human.head().round(2)

,Zone,Disponibilité_intérieure_milliers_tonnes,Nourriture,Proportion Nourriture
0,Afghanistan,13515.0,10735.0,0.79
1,Afrique du Sud,61256.0,29812.0,0.49
2,Albanie,4758.0,3476.0,0.73
3,Algérie,42630.0,31729.0,0.74
4,Allemagne,162275.0,79238.0,0.49


In [43]:
# Countries that allocated the most domestic availability to human consumption
top_human_countries = avail_prop_human.sort_values("Proportion Nourriture", ascending=False)
top_human_countries.reset_index(drop=True).head().round(2)

,Zone,Disponibilité_intérieure_milliers_tonnes,Nourriture,Proportion Nourriture
0,Chine - RAS de Macao,383.0,368.0,0.96
1,Timor-Leste,447.0,419.0,0.94
2,Yémen,9782.0,9162.0,0.94
3,Lesotho,966.0,899.0,0.93
4,Rwanda,10054.0,9273.0,0.92


### Section 7: Proportion of Domestic Availability Wasted<a class="anchor" id="section_4_5"></a>

### 7.1 Total Proportion of Domestic Availability wasted<a class="anchor" id="sub_section_4_5_2"></a>

In [44]:
total_wasted_quantity = food_availability["Pertes"].sum()
print("The quantity of domestic availability wasted is", round(total_wasted_quantity), "thousand tonnes.")

The quantity of domestic availability wasted is 453698 thousand tonnes.


In [45]:
proportion_wasted = (total_wasted_quantity / global_domestic_availability) * 100
print("The proportion of allocated domestic availability wasted is", round(proportion_wasted, 2), "%")

The proportion of allocated domestic availability wasted is 4.6 %


### 7.2 Proportion of Domestic Availability wasted, by Country<a class="anchor" id="sub_section_4_5_1"></a>

In [46]:
wasted_by_country = food_availability[food_availability["Pertes"] > 0]
wasted_by_country = wasted_by_country.groupby(["Zone"])["Pertes"].sum().reset_index()

In [47]:
# Merge domestic availability by country with losses by country
prop_wastes = pd.merge(country_domestic_availability, wasted_by_country, how="left", on="Zone")
prop_wastes.reset_index(drop=True).head().round(2)

,Zone,Disponibilité_intérieure_milliers_tonnes,Pertes
0,Afghanistan,13515.0,1135.0
1,Afrique du Sud,61256.0,2193.0
2,Albanie,4758.0,276.0
3,Algérie,42630.0,3753.0
4,Allemagne,162275.0,3781.0


In [48]:
prop_wastes["Proportion Pertes"] = prop_wastes["Pertes"] / prop_wastes["Disponibilité_intérieure_milliers_tonnes"]
prop_wastes.head().round(2)

,Zone,Disponibilité_intérieure_milliers_tonnes,Pertes,Proportion Pertes
0,Afghanistan,13515.0,1135.0,0.08
1,Afrique du Sud,61256.0,2193.0,0.04
2,Albanie,4758.0,276.0,0.06
3,Algérie,42630.0,3753.0,0.09
4,Allemagne,162275.0,3781.0,0.02


In [49]:
# Countries losing most of their domestic food availability
top_waste_countries = prop_wastes.sort_values("Proportion Pertes", ascending=False)
top_waste_countries.reset_index(drop=True).head().round(2)

,Zone,Disponibilité_intérieure_milliers_tonnes,Pertes,Proportion Pertes
0,Sierra Leone,6570.0,2191.0,0.33
1,Ghana,38998.0,7442.0,0.19
2,Cameroun,26222.0,3981.0,0.15
3,Angola,32029.0,4799.0,0.15
4,Dominique,141.0,20.0,0.14


# Part 2 - Malnutrition by Country

### Section 1: Countries with the Highest Proportion of Malnourished People in 2017<a class="anchor" id="question5"></a>

In [50]:
# Calculate the proportion of malnourished people by country
malnourished_merged["Malnutrition (%)"] = (malnourished_merged["Population_sous_alimentée"] / malnourished_merged["Population_totale"]) * 100
display(malnourished_merged.head().round(2))

,Zone,Population_totale,Population_sous_alimentée,Malnutrition (%)
0,Afghanistan,36.30,10.5,28.93
1,Afrique du Sud,57.01,3.1,5.44
2,Albanie,2.88,0.1,3.47
3,Algérie,41.39,1.3,3.14
4,Allemagne,82.66,0.0,0.00


In [51]:
# Get the top 10 countries with the highest proportion of malnourished people
malnourished_merged_top10 = malnourished_merged.sort_values("Malnutrition (%)", ascending=False).reset_index(drop=True).head(10)
display(malnourished_merged_top10[["Zone", "Population_totale", "Population_sous_alimentée", "Malnutrition (%)"]].round(2))

,Zone,Population_totale,Population_sous_alimentée,Malnutrition (%)
0,Haïti,10.98,5.3,48.26
1,République populaire démocratique de Corée,25.43,12.0,47.19
2,Madagascar,25.57,10.5,41.06
3,Libéria,4.70,1.8,38.28
4,Lesotho,2.09,0.8,38.25
5,Tchad,15.02,5.7,37.96
6,Rwanda,11.98,4.2,35.06
7,Mozambique,28.65,9.4,32.81
8,Timor-Leste,1.24,0.4,32.17
9,Afghanistan,36.30,10.5,28.93


### Section 2: Countries Benefiting from Donations since 2013<a class="anchor" id="question6"></a>

### 2.1 Amounts of Donations since 2013<a class="anchor" id="section_6_1"></a>

In [52]:
donations_2013 = donations_2013[donations_2013["Quantité"] > 0]
total_donations = donations_2013["Quantité"].sum()
print("Since 2013, the amount of food donations has been", round(total_donations), "thousand tons.")

Since 2013, the amount of food donations has been 4166 thousand tons.


### 2..2 Amounts of Donations, by Country<a class="anchor" id="section_6_2"></a>

In [53]:
# Group donations data by country
donations_2013_country = donations_2013.groupby(["Zone"])["Quantité"].sum().reset_index()

In [54]:
# Display the top 10 countries benefiting from the most donations since 2013
donations_2013_country_top_10 = donations_2013_country.sort_values("Quantité", ascending=False).reset_index(drop=True).head(10)
display(donations_2013_country_top_10.round(2))

,Zone,Quantité
0,Éthiopie,591.40
1,République arabe syrienne,563.57
2,Soudan,330.23
3,Yémen,264.76
4,Kenya,220.97
5,Soudan du Sud,196.33
6,République démocratique du Congo,150.32
7,Somalie,139.80
8,Bangladesh,131.02
9,Afghanistan,128.24


In [55]:
# Merge data on malnourished population and donations by country
donations_malnourished_pop = pd.merge(malnourished_merged_top10, donations_2013_country_top_10, how="left", on="Zone")
display(donations_malnourished_pop.round(2))

,Zone,Population_totale,Population_sous_alimentée,Malnutrition (%),Quantité
0,Haïti,10.98,5.3,48.26,NaN
1,République populaire démocratique de Corée,25.43,12.0,47.19,NaN
2,Madagascar,25.57,10.5,41.06,NaN
3,Libéria,4.70,1.8,38.28,NaN
4,Lesotho,2.09,0.8,38.25,NaN
5,Tchad,15.02,5.7,37.96,NaN
6,Rwanda,11.98,4.2,35.06,NaN
7,Mozambique,28.65,9.4,32.81,NaN
8,Timor-Leste,1.24,0.4,32.17,NaN
9,Afghanistan,36.30,10.5,28.93,128.24


### 3.1 Maximum and Minimum Food Availability per Inhabitant<a class="anchor" id="question7"></a>

In [56]:
# Merge population by country with food availability by country
avail_per_inhab = pd.merge(country_domestic_availability, population_2017, how="right", on="Zone")

In [73]:
avail_per_inhab = avail_per_inhab[avail_per_inhab["Disponibilité_intérieure_milliers_tonnes"] > 0]
avail_per_inhab["Availability per Capita"] = avail_per_inhab["Disponibilité_intérieure_milliers_tonnes"] / avail_per_inhab["Population_totale"]
display(avail_per_inhab.head().round(2))

,Zone,Disponibilité_intérieure_milliers_tonnes,Année,Population_totale,Availability per Capita
0,Afghanistan,13515.0,2017,36.30,372.35
1,Afrique du Sud,61256.0,2017,57.01,1074.48
2,Albanie,4758.0,2017,2.88,1649.70
3,Algérie,42630.0,2017,41.39,1029.98
4,Allemagne,162275.0,2017,82.66,1963.20


In [58]:
# Get the top 10 countries with the highest food availability per inhabitant
avail_per_inhab_top10 = avail_per_inhab.sort_values("Availability per Capita", ascending=False).reset_index(drop=True).head(10)
display(avail_per_inhab_top10.round(2))

,Zone,Disponibilité_intérieure_milliers_tonnes,Année,Population_totale,Availability per Capita
0,Eswatini,6467.0,2017,1.12,5749.44
1,Brésil,1033314.0,2017,207.83,4971.83
2,Guyana,3290.0,2017,0.78,4243.95
3,Belize,1542.0,2017,0.38,4103.58
4,Maurice,4767.0,2017,1.26,3769.87
5,Danemark,19749.0,2017,5.73,3445.23
6,Bélarus,27522.0,2017,9.45,2912.31
7,Fidji,2499.0,2017,0.88,2848.00
8,Australie,66992.0,2017,24.58,2724.96
9,Lituanie,7467.0,2017,2.85,2624.22


In [59]:
# Get the bottom 10 countries with the lowest food availability per inhabitant
avail_per_inhab_flop10 = avail_per_inhab.sort_values("Availability per Capita", ascending=True).reset_index(drop=True).head(10)
display(avail_per_inhab_flop10.round(2))

,Zone,Disponibilité_intérieure_milliers_tonnes,Année,Population_totale,Availability per Capita
0,Yémen,9782.0,2017,27.83,351.43
1,Tchad,5304.0,2017,15.02,353.21
2,Timor-Leste,447.0,2017,1.24,359.54
3,Afghanistan,13515.0,2017,36.30,372.35
4,Éthiopie,44014.0,2017,106.40,413.67
5,Gambie,940.0,2017,2.21,424.59
6,Libéria,2002.0,2017,4.70,425.76
7,Guinée-Bissau,786.0,2017,1.83,429.94
8,Bangladesh,72757.0,2017,159.69,455.63
9,Lesotho,966.0,2017,2.09,461.86


# Additional Information

###  Section 1: Use of Cereals<a class="anchor" id="question8"></a>

In [60]:
# Filter cereals (wheat, corn, rye, barley, rice, oats, millet, sorghum)
cereal_products = ["Céréales, Autres", "Blé", "Avoine", "Orge", "Seigle", "Maïs", "Millet", "Riz (Eq Blanchi)", "Sorgho"]
avail_cereals = food_availability[food_availability["Produit"].isin(cereal_products)]
avail_cereals["Catégorie"] = "céréales"
avail_cereals = avail_cereals[["Zone", "Origine", "Catégorie", "Disponibilité_intérieure_milliers_tonnes", "Nourriture", "Aliments pour animaux"]]

In [61]:
# Display the data
display(avail_cereals.head().round(2))

,Zone,Origine,Catégorie,Disponibilité_intérieure_milliers_tonnes,Nourriture,Aliments pour animaux
7,Afghanistan,vegetale,céréales,5992.0,4895.0,NaN
12,Afghanistan,vegetale,céréales,0.0,0.0,NaN
32,Afghanistan,vegetale,céréales,313.0,76.0,200.0
34,Afghanistan,vegetale,céréales,13.0,12.0,NaN
40,Afghanistan,vegetale,céréales,524.0,89.0,360.0


### 1.1 Proportion of Cereals Worldwide<a class="anchor" id="section_8_1"></a>

In [62]:
total_cereals_quantity = avail_cereals["Disponibilité_intérieure_milliers_tonnes"].sum()
print("The quantity of cereals is", round(total_cereals_quantity), "thousand tonnes.")

The quantity of cereals is 2406999 thousand tonnes.


In [63]:
cereals_prop_global = (total_cereals_quantity / global_domestic_availability) * 100
print("Cereals account for", round(cereals_prop_global, 2), "% of the world's domestic food availability.")

Cereals account for 24.43 % of the world's domestic food availability.


### 1.2 Proportion of Cereals allocated to Human Consumption in relation to Total Domestic Availability<a class="anchor" id="sub_section_8_1_2"></a>

In [64]:
total_cereals_human = avail_cereals["Nourriture"].sum()
avail_global_cereals_human = (total_cereals_human / total_cereals_quantity) * 100
print("The proportion of cereals availability allocated to human consumption is", round(avail_global_cereals_human, 2), "%")

The proportion of cereals availability allocated to human consumption is 42.75 %


### 1.3 Proportion of Cereals allocated to Animal Nutrition in relation to Total Domestic Availability<a class="anchor" id="sub_section_8_1_3"></a>

In [65]:
total_cereals_animal = avail_cereals["Aliments pour animaux"].sum()
avail_global_cereals_animal = (total_cereals_animal / total_cereals_quantity) * 100
print("The proportion of cereals availability allocated to animal nutrition is", round(avail_global_cereals_animal, 2), "%")

The proportion of cereals availability allocated to animal nutrition is 36.29 %


### Section 2: Use of Cereals by Country

### 2.1 Quantity of Cereals, by Country<a class="anchor" id="section_8_2"></a>

In [66]:
avail_cereals_country = avail_cereals.groupby("Zone")["Disponibilité_intérieure_milliers_tonnes"].sum().reset_index()
display(avail_cereals_country.head().round(2))

,Zone,Disponibilité_intérieure_milliers_tonnes
0,Afghanistan,7303.0
1,Afrique du Sud,15493.0
2,Albanie,1145.0
3,Algérie,15013.0
4,Allemagne,42158.0


### 2.2 Quantity of cereals for Human Consumption, by Country

In [67]:
avail_cereals_human_country = avail_cereals[avail_cereals["Nourriture"] > 0].groupby("Zone")["Nourriture"].sum().reset_index()
display(avail_cereals_human_country.head().round(2))

,Zone,Nourriture
0,Afghanistan,5494.0
1,Afrique du Sud,9503.0
2,Albanie,474.0
3,Algérie,8535.0
4,Allemagne,9191.0


### 2.3 Quantity of Cereals allocated to Animal Nutritrion, by Country

In [68]:
avail_cereals_animal_country = avail_cereals[avail_cereals["Aliments pour animaux"] > 0].groupby("Zone")["Aliments pour animaux"].sum().reset_index()
display(avail_cereals_animal_country.head().round(2))

,Zone,Aliments pour animaux
0,Afghanistan,560.0
1,Afrique du Sud,4824.0
2,Albanie,411.0
3,Algérie,4337.0
4,Allemagne,26615.0


### 2.4 Proportion of Cereals Availability, by Country

In [74]:
dispo_cereals_all = pd.merge(pd.merge(country_domestic_availability, avail_cereals_animal_country, how="left", on="Zone"), avail_cereals_human_country, how="left", on="Zone")
dispo_cereals_all = dispo_cereals_all.rename(columns={"Disponibilité_intérieure_milliers_tonnes": "Céréales (milliers de tonnes)", "Aliments pour animaux": "Céréales pour les animaux (milliers de tonnes)", "Nourriture": "Céréales pour l'homme (milliers de tonnes)"})
display(dispo_cereals_all.head().round(2))

,Zone,Céréales (milliers de tonnes),Céréales pour les animaux (milliers de tonnes),Céréales pour l'homme (milliers de tonnes)
0,Afghanistan,13515.0,560.0,5494.0
1,Afrique du Sud,61256.0,4824.0,9503.0
2,Albanie,4758.0,411.0,474.0
3,Algérie,42630.0,4337.0,8535.0
4,Allemagne,162275.0,26615.0,9191.0


### 2.5 Proportion of Cereals Availability by Country in relation to Total Food Availability

In [75]:
dispo_cereals_all["Proportion pour les animaux (%)"] = (dispo_cereals_all["Céréales pour les animaux (milliers de tonnes)"] / dispo_cereals_all["Céréales (milliers de tonnes)"]) * 100
dispo_cereals_all["Proportion pour les humains (%)"] = (dispo_cereals_all["Céréales pour l'homme (milliers de tonnes)"] / dispo_cereals_all["Céréales (milliers de tonnes)"]) * 100

In [77]:
# Display countries with the highest proportion of cereals used in animal nutrition
top_cereals_animal = dispo_cereals_all.sort_values("Proportion pour les animaux (%)", ascending=False).reset_index(drop=True).head()
top_cereals_animal = top_cereals_animal[["Zone", "Céréales (milliers de tonnes)", "Proportion pour les animaux (%)", "Proportion pour les humains (%)"]]
display(top_cereals_animal.round(2))

,Zone,Céréales (milliers de tonnes),Proportion pour les animaux (%),Proportion pour les humains (%)
0,Arabie saoudite,31274.0,37.35,14.19
1,Danemark,19749.0,33.67,3.73
2,Canada,79562.0,30.05,5.28
3,Serbie,18156.0,27.30,7.03
4,Irlande,10738.0,26.64,5.44


In [78]:
# Display countries with the highest proportion of cereals used in human consumption
top_cereals_human = dispo_cereals_all.sort_values("Proportion pour les humains (%)", ascending=False).reset_index(drop=True).head()
top_cereals_human = top_cereals_human[["Zone", "Céréales (milliers de tonnes)", "Proportion pour les animaux (%)", "Proportion pour les humains (%)"]]
display(top_cereals_human.round(2))

,Zone,Céréales (milliers de tonnes),Proportion pour les animaux (%),Proportion pour les humains (%)
0,Lesotho,966.0,0.10,48.55
1,Timor-Leste,447.0,1.79,41.16
2,Bangladesh,72757.0,3.75,40.91
3,Afghanistan,13515.0,4.14,40.65
4,Yémen,9782.0,2.15,40.38
